## Siamese

In [1]:
import os
os.chdir('../../src/')

In [2]:
import torch
import torch.utils.data as data
from PIL import Image, ImageOps
import pickle
import numpy as np
from torchvision import transforms
import json
from credential_classifier.bit_pytorch.dataloader import HybridLoader
from credential_classifier.bit_pytorch.models import KNOWN_MODELS
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"]="1"

- Define dataloader

In [3]:
val_set = HybridLoader(img_folder='../datasets/val_merge_imgs',
                         annot_path='../datasets/val_merge_coords.txt')

valid_loader = torch.utils.data.DataLoader(
  val_set, batch_size=8, shuffle=False, pin_memory=True, drop_last=False)

In [4]:
len(valid_loader)

467

- Accuracy function

In [15]:
def compute_acc(dataloader, model, device):
    correct = 0
    total = 0

    for b, (x, y) in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            x = x.to(device, non_blocking=True, dtype=torch.float)
            y = y.to(device, non_blocking=True)
            logits = model(x)
            pred_cls = torch.argmax(logits, dim=1)

            correct += torch.sum(torch.eq(pred_cls, y)).item()
            total += y.shape[0]
            
    print('Accuracy after changing relu function: {:.2f}'.format(correct/total))    
    return correct/total

- Load model (original)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model
model = KNOWN_MODELS["BiT-M-R50x1V2"](head_size=2, zero_head=True)

# Load weights
checkpoint = torch.load('/home/l/liny/ruofan/PhishIntention/src/credential_classifier/output/hybrid/hybrid_lr0.005/BiT-M-R50x1V2_0.005.pth.tar', 
                        map_location="cpu")["model"]

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)

model.to(device)
model.eval()

ResNetV2Coord(
  (root): Sequential(
    (conv): StdConv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (block1): Sequential(
      (unit01): PreActBottleneck(
        (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
        (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (unit02): PreActBottleneck(
        (gn1): GroupNorm(32, 256, eps=1e-05

In [16]:
compute_acc(valid_loader, model, device)

3731it [11:43,  5.31it/s]

Accuracy after changing relu function: 0.95


0.9471991423210936

## Load model (change activation function)

In [5]:
import torch
import torch.nn.functional as F
from torch import nn

In [6]:
class QuantizeRelu(nn.Module):
    def __init__(self, step_size = 0.01):
        super().__init__()
        self.step_size = step_size

    def forward(self, x):
        mask = torch.ge(x, 0).bool() # mask for positive values
        quantize = torch.ones_like(x) * self.step_size
        out = torch.mul(torch.floor(torch.div(x, quantize)), self.step_size) # quantize by step_size
        out = torch.mul(out, mask) # zero-out negative values
        out = torch.abs(out) # remove sign
        return out

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model
model = KNOWN_MODELS["BiT-M-R50x1V2"](head_size=2, zero_head=True)
# Load weights
checkpoint = torch.load('/home/l/liny/ruofan/PhishIntention/src/credential_classifier/output/hybrid/hybrid_lr0.005/BiT-M-R50x1V2_0.005.pth.tar', 
                        map_location="cpu")["model"]

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)

# replace relu with defenselayer 
# model.body.block1.unit01.relu = QuantizeRelu()
# model.body.block1.unit02.relu = QuantizeRelu()
# model.body.block1.unit03.relu = QuantizeRelu()

# model.body.block2.unit01.relu = QuantizeRelu()
# model.body.block2.unit02.relu = QuantizeRelu()
# model.body.block2.unit03.relu = QuantizeRelu()
# model.body.block2.unit04.relu = QuantizeRelu()

# model.body.block3.unit01.relu = QuantizeRelu()
# model.body.block3.unit02.relu = QuantizeRelu()
# model.body.block3.unit03.relu = QuantizeRelu()
# model.body.block3.unit04.relu = QuantizeRelu()
# model.body.block3.unit05.relu = QuantizeRelu()
# model.body.block3.unit06.relu = QuantizeRelu()

model.body.block4.unit01.relu = QuantizeRelu()
model.body.block4.unit02.relu = QuantizeRelu()
model.body.block4.unit03.relu = QuantizeRelu()

model.to(device)
model.eval()

ResNetV2Coord(
  (root): Sequential(
    (conv): StdConv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (block1): Sequential(
      (unit01): PreActBottleneck(
        (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
        (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (unit02): PreActBottleneck(
        (gn1): GroupNorm(32, 256, eps=1e-05

In [19]:
# compute_acc(valid_loader, model, device)

Accuracy after changing relu function: 0.93


0.9345417925478349

## Attack

In [11]:
from adv_attack.attack.Attack import *

criterion = nn.CrossEntropyLoss()
check = adversarial_attack(method='fgsm', model=model, dataloader=valid_loader, 
                           device=device, num_classes=2, save_data=True)

acc, _ = check.batch_attack()

3558it [6:33:19,  6.94s/it]

3557
Test Accuracy = 0.9471613265879708


3559it [6:33:25,  6.84s/it]

3558
Test Accuracy = 0.9471761730823265


3560it [6:33:33,  6.94s/it]

...

3728


3730it [6:52:27,  5.61s/it]

3729
Test Accuracy = 0.9471849865951742


3731it [6:52:35,  6.64s/it]

3730
Test Accuracy = 0.9471991423210936
Test Accuracy = 3534 / 3731 = 0.9471991423210936


In [12]:
acc

0.9471991423210936

## BPDA

- With defense

In [7]:
from advertorch.bpda import BPDAWrapper
from advertorch.attacks import LinfPGDAttack

In [8]:
# original model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model
model = KNOWN_MODELS["BiT-M-R50x1V2"](head_size=2, zero_head=True)
# Load weights
checkpoint = torch.load('/home/l/liny/ruofan/PhishIntention/src/credential_classifier/output/hybrid/hybrid_lr0.005/BiT-M-R50x1V2_0.005.pth.tar', 
                        map_location="cpu")["model"]

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
model.to(device)
model.eval()

# BPDA approximation of gradients
defense_layer = BPDAWrapper(QuantizeRelu())

# # replace relu with defenselayer 
model.body.block1.unit01.relu = defense_layer
model.body.block1.unit02.relu = defense_layer
model.body.block1.unit03.relu = defense_layer

model.body.block2.unit01.relu = defense_layer
model.body.block2.unit02.relu = defense_layer
model.body.block2.unit03.relu = defense_layer
model.body.block2.unit04.relu = defense_layer

model.body.block3.unit01.relu = defense_layer
model.body.block3.unit02.relu = defense_layer
model.body.block3.unit03.relu = defense_layer
model.body.block3.unit04.relu = defense_layer
model.body.block3.unit05.relu = defense_layer
model.body.block3.unit06.relu = defense_layer

model.body.block4.unit01.relu = defense_layer
model.body.block4.unit02.relu = defense_layer
model.body.block4.unit03.relu = defense_layer

bpda_adversary = LinfPGDAttack(
    model, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.05,
    nb_iter=100, eps_iter=0.005, rand_init=True, clip_min=0.0, clip_max=1.0,
    targeted=False)


In [ ]:
perturb_correct = 0
total = 0

for cln_data, true_label in tqdm(valid_loader):
    cln_data, true_label = cln_data.to(device, dtype=torch.float), true_label.to(device)
    bpda_adv = bpda_adversary.perturb(cln_data, true_label)
    
    logits = model(bpda_adv)
    pred_cls = torch.argmax(logits, dim=1)
    perturb_correct += torch.sum(torch.eq(pred_cls, true_label)).item()
    total += len(true_label)
    
    print(perturb_correct/total)

  0%|          | 1/467 [00:17<2:12:18, 17.03s/it]

0.875


  0%|          | 2/467 [00:33<2:10:46, 16.87s/it]

0.9375


  1%|          | 3/467 [00:50<2:10:06, 16.82s/it]

...

0.8914473684210527


 41%|████      | 191/467 [52:06<1:13:36, 16.00s/it]

0.8913612565445026


 41%|████      | 192/467 [52:22<1:13:14, 15.98s/it]

0.8912760416666666


In [14]:
perturb_correct/total

0.0002680246582685607

In [24]:
perturb_correct/total

0.905337361530715

In [14]:
torch.randint(0, 277, (1,))

tensor([203])